In [1]:
!which python

/Users/darrenkwon/statistics-and-data-driven/.venv/bin/python


In [3]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pylab as plt

In [11]:
from empiricaldist import Pmf

coin = Pmf()
coin["heads"] = 0.5
coin["tails"] = 0.5

die = Pmf.from_seq(range(1, 7))
die

,probs
1,0.166667
2,0.166667
3,0.166667
4,0.166667
5,0.166667
6,0.166667


In [23]:
# cookie problem with Pmf

prior = Pmf.from_seq(['Bowl 1', 'Bowl 2'])
likelihood_vanilla = [0.75, 0.5]
posterior = prior * likelihood_vanilla

# Make the probabilities add up to 1 (modifies self).
# 더해서 1이 되게 하려면, Pmf에서 제공하는 normalize()를 사용하면 된다.
# 아래는 posterior를 normalize하였다. 
posterior.normalize() 
np.sum(posterior) # 이제 1이다!

posterior

,probs
Bowl 1,0.6
Bowl 2,0.4


## 반복 시행을 통한 likelihood의 갱신

posterior1 = prior * likelihood_vanilla
posterior2 = posterior1 * likelihood_vanilla
posterior3 = posterior2 * likelihood_chocolate
...

In [32]:
# 101 cookies problem with Pmf

hypos = np.arange(101)
prior = Pmf(1, hypos) # # 확률, 사건. -> 모두 동일 확률이므로 일단은 1로 초기화
prior.normalize() # normalize sum up to 1. 모두 1로 초기화 했는데 합이 1이 되어야 하므로 -> 1 / 전체 갯수 가 될 것이다.

likelihood_vanilla = hypos / 100
posterior1 = prior * likelihood_vanilla # unniformed posterior
posterior1.normalize()
posterior1.head()

posterior2 = posterior1 * likelihood_vanilla # 한 번 더 바닐라를 뽑았으므로 사후확률에 likelihood를 한 번 더 곱하면 됨
posterior2.normalize()

likelihood_chocolate = 1 - hypos/100

posterior3 = posterior2 * likelihood_chocolate
posterior3.normalize()

posterior3.head()

0      0.009901
1      0.009901
2      0.009901
3      0.009901
4      0.009901
         ...   
96     0.009901
97     0.009901
98     0.009901
99     0.009901
100    0.009901
Name: , Length: 101, dtype: float64


,probs
0,0.000000
1,0.000012
2,0.000047


In [30]:
# MAP = maximum a posteori probability (사후 확률에서 가장 높은 값)

# 아래 둘은 같은 기능.
posterior3.idxmax()
posterior3.max_prob()

67

In [36]:
# die problem with Pmf

hypos = [6, 8, 12] # 가능한 주사위 면 수
prior = Pmf(1/3, hypos) # 사전 확률. 

prior.qs, prior.ps # 각각 가능한 원소와 그 원소에 대한 확률

# 1이 나왔을 때의 우도
likelihood1 = 1/6, 1/8, 1/12 
posterior = prior * likelihood1
posterior.normalize()
posterior

# 7이 나왔을 때의 우도
likelihood2 = 0, 1/8, 1/12
posterior *= likelihood2
posterior.normalize()
posterior


,probs
6,0.000000
8,0.692308
12,0.307692


In [37]:
def update_dice(pmf, data):
    """Update pmf based on new data."""
    hypos = pmf.qs
    likelihood = 1 / hypos
    impossible = (data > hypos)
    likelihood[impossible] = 0
    pmf *= likelihood
    pmf.normalize()